# Calculation of Spectrum Averaged Cross Sections

In this notebook we perform a fit with GMAP and then compute posterior estimates of Spectrum Averaged Cross Sections (SACS) and their uncertainties using the sandwich formula. We will make two GMAP evaluations: One will be in legacy mode with the PPP bug and SACS Jacobian bug present and the other one without these bugs. Both calculations will be done with the PPP correction enabled (i.e., percentual experimental uncertainties are interpreted as being relative to the fitted cross sections).

First we load required packages:

In [1]:
import numpy as np
import pandas as pd
from gmapi.gmap import run_gmap

We use a database file from the test data. This file is also the basis for the neutron standards evaluation 2017.

In [2]:
dbfile = '../tests/testdata/data-2017-07-26.gma'

The next lines perform the calculations, the first one in legacy mode and the other one in modern mode.

In [3]:
%%capture 
res1 = run_gmap(dbfile, dbtype='legacy', num_iter=3, correct_ppp=True,
                fix_ppp_bug=False, fix_sacs_jacobian=False, legacy_output=False)

res2 = run_gmap(dbfile, dbtype='legacy', num_iter=3, correct_ppp=True,
                fix_ppp_bug=True, fix_sacs_jacobian=True, legacy_output=False)

*Note*: The `%%capture` statement is there to discard the output of this cell, which would otherwise produce a warning about a division by zero caused by some experimental dataset.

## Construction of a synthetic experiment table 

SACS values and their uncertainties are not estimated by the GMAP functions above but some SACS measurements are nevertheless used as constraints during inference. To obtain posterior estimates and uncertainties of the SACS values, we need to construct a synthetic experiment table and propagate the posterior estimates of the cross sections manually to the SACS values.

To construct a synthetic experiment table, let us remind ourselves of the structure of an experiment table by inspecting the one used in the fits above. We select only SACS measurements to obtain a concise table.

In [4]:
sacs_exptable = res1['exptable'][res1['exptable'].REAC.str.match('MT:6-')].reset_index()
sacs_exptable

,index,NODE,REAC,ENERGY,DATA,DB_IDX,DS_IDX
0,1290,exp_565,MT:6-R1:8,0.15,1.215,55,2
1,1291,exp_641,MT:6-R1:9,1.50,1.790,55,3
2,1293,exp_575,MT:6-R1:8,1.20,1.254,55,5
3,1294,exp_576,MT:6-R1:8,1.20,1.216,55,6
4,1295,exp_674,MT:6-R1:9,1.20,1.824,55,7
5,1296,exp_517,MT:6-R1:8,1.50,1.234,55,8
6,1297,exp_614,MT:6-R1:9,1.50,1.844,55,9


The relevant columns that must be present for a mapping are `NODE`, `REAC`, and `ENERGY`. The `DATA` column contains the SACS measurements and is only used for fitting. Let's augment this table by the reaction string in human-readable form.

In [5]:
posttable1 = res1['table']
posttable2 = res2['table']
reac_descr = [posttable1[posttable1.REAC == p].DESCR.iloc[0]
              for p in sacs_exptable['REAC'].str.replace('MT:6-','MT:1-').tolist()]
sacs_exptable['DESCR'] = reac_descr
sacs_exptable

,index,NODE,REAC,ENERGY,DATA,DB_IDX,DS_IDX,DESCR
0,1290,exp_565,MT:6-R1:8,0.15,1.215,55,2,"U5(n,f)"
1,1291,exp_641,MT:6-R1:9,1.50,1.790,55,3,"PU9(n,f)"
2,1293,exp_575,MT:6-R1:8,1.20,1.254,55,5,"U5(n,f)"
3,1294,exp_576,MT:6-R1:8,1.20,1.216,55,6,"U5(n,f)"
4,1295,exp_674,MT:6-R1:9,1.20,1.824,55,7,"PU9(n,f)"
5,1296,exp_517,MT:6-R1:8,1.50,1.234,55,8,"U5(n,f)"
6,1297,exp_614,MT:6-R1:9,1.50,1.844,55,9,"PU9(n,f)"


This table shows that several measurements of the same SACS quantity have been made. For the prediction, we keep the unique reactions only.

In [6]:
syntable = sacs_exptable.groupby('REAC').first().reset_index().drop(['index', 'DB_IDX', 'DS_IDX', 'DATA'], axis=1)
syntable

,REAC,NODE,ENERGY,DESCR
0,MT:6-R1:8,exp_565,0.15,"U5(n,f)"
1,MT:6-R1:9,exp_641,1.50,"PU9(n,f)"


## Calculation of posterior SACS values and uncertainties

Now let's propagate the estimates of the basic cross sections to obtain the SACS values.

In [7]:
from gmapi.mappings.compound_map import CompoundMap
compmap = CompoundMap(fix_sacs_jacobian=True)
predvals1 = compmap.propagate(posttable1, syntable, posttable1.POST.to_numpy())
predvals2 = compmap.propagate(posttable2, syntable, posttable2.POST.to_numpy())
syntable['PRED1'] = predvals1
syntable['PRED2'] = predvals2
syntable['NODE'] = ['sacs1', 'sacs2']
syntable

,REAC,NODE,ENERGY,DESCR,PRED1,PRED2
0,MT:6-R1:8,sacs1,0.15,"U5(n,f)",1.224776,1.225197
1,MT:6-R1:9,sacs2,1.50,"PU9(n,f)",1.805102,1.806380


We also want posterior uncertainties for the SACS values. We obtain them by relying on the sensitivity matrix  and the posterior covariance matrix. 

In [8]:
S1 = compmap.jacobian(posttable1, syntable, posttable1.POST.to_numpy(), ret_mat=True)
postcov1 = res1['postcov']
sacs_unc1 = np.sqrt(np.diag(np.array(S1 @ postcov1 @ S1.T)))

S2 = compmap.jacobian(posttable2, syntable, posttable2.POST.to_numpy(), ret_mat=True)
postcov2 = res2['postcov']
sacs_unc2 = np.sqrt(np.diag(np.array(S2 @ postcov2 @ S2.T)))

syntable['UNC1'] = sacs_unc1
syntable['UNC2'] = sacs_unc2
syntable

,REAC,NODE,ENERGY,DESCR,PRED1,PRED2,UNC1,UNC2
0,MT:6-R1:8,sacs1,0.15,"U5(n,f)",1.224776,1.225197,0.003466,0.003468
1,MT:6-R1:9,sacs2,1.50,"PU9(n,f)",1.805102,1.806380,0.006056,0.006056


Finally, we compare this to the experiment table:

In [9]:
sacs_exptable.sort_values('REAC')

,index,NODE,REAC,ENERGY,DATA,DB_IDX,DS_IDX,DESCR
0,1290,exp_565,MT:6-R1:8,0.15,1.215,55,2,"U5(n,f)"
2,1293,exp_575,MT:6-R1:8,1.20,1.254,55,5,"U5(n,f)"
3,1294,exp_576,MT:6-R1:8,1.20,1.216,55,6,"U5(n,f)"
5,1296,exp_517,MT:6-R1:8,1.50,1.234,55,8,"U5(n,f)"
1,1291,exp_641,MT:6-R1:9,1.50,1.790,55,3,"PU9(n,f)"
4,1295,exp_674,MT:6-R1:9,1.20,1.824,55,7,"PU9(n,f)"
6,1297,exp_614,MT:6-R1:9,1.50,1.844,55,9,"PU9(n,f)"
